In [11]:
#%% 0. 环境准备
# 安装依赖（Notebook 中使用 !pip install）
# !pip install langgraph langchain openai python-dotenv networkx matplotlib langsmith

In [57]:
#%% 1. 导入库
import os
from typing import TypedDict, List

from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import rcParams
from datetime import datetime
from IPython.display import display, Markdown
from langchain_core.tracers import LangChainTracer
from langsmith import traceable

#%% 2. 环境设置与全局配置
rcParams['font.sans-serif'] = ['Heiti TC']
rcParams['axes.unicode_minus'] = False
load_dotenv()

#%% 3. 定义对话状态结构
class AgentState(TypedDict):
    messages: List[BaseMessage] # 对话历史
    agent_outcome: str # 下一步决策：tool 或 final
    tool_response: str # 工具调用结果

#%% 4. 初始化大模型
llm = ChatOpenAI(model_name="deepseek-chat", verbose=True)

#%% 5. 节点函数定义
def agent_decision_node(state: AgentState) -> dict:
    last_message = state["messages"][-1].content
    prompt = ChatPromptTemplate.from_template(
        "请判断用户的意图：\n"
        "用户输入：{input}\n\n"
        "如果需要调用工具，请回答 tool；否则回答 final。"
    )
    decision = llm.invoke(prompt.format_messages(input=last_message)).content.strip().lower()
    print(f"\n[Agent Decision] outcome = {decision}")
    return {"agent_outcome": decision if decision in ("tool", "final") else "final"}

def tool_node(_: AgentState) -> dict:
    return {"tool_response": "明天晴，气温25°C，适合出行。"}

def final_node(state: AgentState) -> dict:
    # 获取工具的响应，或者默认消息
    reply = state.get("tool_response", "很高兴为您服务！")
    
    # 获取当前时间戳，作为元数据
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # 可以添加额外的元数据，例如，用户的原始输入
    user_input = state["messages"][-1].content
    
    # 打印出回复信息及其附加内容
    print(f"回复: {reply}, 时间: {timestamp}, 用户输入: {user_input}")
    
    # 返回包含原始消息和回复的消息记录
    return {
        "messages": state["messages"] + [AIMessage(content=reply)],
        "metadata": {
            "timestamp": timestamp,
            "user_input": user_input,
            "reply_length": len(reply),
        }
    }

#%% 6. 构建 LangGraph 工作流
def build_graph():
    workflow = StateGraph(AgentState)
    workflow.add_node("agent", agent_decision_node)
    workflow.add_node("tool", tool_node)
    workflow.add_node("final", final_node)
    workflow.set_entry_point("agent")
    workflow.set_finish_point("final")
    workflow.add_conditional_edges("agent", lambda st: st["agent_outcome"], {"tool": "tool", "final": "final"})
    workflow.add_edge("tool", "final")
    return workflow.compile()

# 编译一次，供全局使用
graph = build_graph()

#%% 7. 运行测试
def test_run():
    init_state = {"messages": [HumanMessage(content="请问明天天气如何？")]}
    result = graph.invoke(init_state)
    print("\n对话记录:")
    for msg in result["messages"]:
        print(f"[{msg.type}]: {msg.content}")

#%% 8. 使用 Mermaid 可视化工作流
def visualize_workflow_with_mermaid(graph):
    mermaid_code = graph.get_graph().draw_mermaid()  # 获取 Mermaid 格式代码
    
    # 通过 IPython 显示 Mermaid 图
    display(Markdown(f"```mermaid\n{mermaid_code}\n```"))

#%% 9. LangSmith 追踪（可选）
def run_with_langsmith(input_state):
    if "LANGSMITH_API_KEY" in os.environ:
        tracer = LangChainTracer()

        print("\n[LangSmith] 开始追踪工作流执行...")
        result = graph.invoke(input_state, config={"callbacks": [tracer]})

        print("\n[Traced] 对话记录:")
        for msg in result["messages"]:
            print(f"[{msg.type}]: {msg.content}")
    else:
        print("\n跳过 LangSmith 集成：未设置 LANGSMITH_API_KEY")

#%% 10. 主入口
if __name__ == "__main__":
    visualize_workflow_with_mermaid(graph)
    # test_run()
    run_with_langsmith({"messages": [HumanMessage(content="北京天气怎么样？")]})

```mermaid
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	agent(agent)
	tool(tool)
	final(final)
	__end__([<p>__end__</p>]):::last
	__start__ --> agent;
	agent -.-> final;
	agent -.-> tool;
	tool --> final;
	final --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

```


[LangSmith] 开始追踪工作流执行...

[Agent Decision] outcome = tool
回复: 明天晴，气温25°C，适合出行。, 时间: 2025-05-05 13:23:00, 用户输入: 北京天气怎么样？

[Traced] 对话记录:
[human]: 北京天气怎么样？
[ai]: 明天晴，气温25°C，适合出行。
